In [ ]:
# OS and Dotenv imports used for environment variable management, keeping API keys secure.
import os
from dotenv import load_dotenv

# Requests library for making HTTP requests to the Strava API.
import requests

# Pretty print for better readability of JSON responses.
from pprint import pprint

Firstly, lets look at the first API call, athlete.

In [ ]:
# Load environment variables from .env file
load_dotenv()
CLIENT_ID = os.getenv("STRAVA_CLIENT_ID")
CLIENT_SECRET = os.getenv("STRAVA_CLIENT_SECRET")
REFRESH_TOKEN = os.getenv("STRAVA_REFRESH_TOKEN")


# Example: Get athlete profile
athlete_url = "https://www.strava.com/api/v3/athlete"

# Authorization header with Access token
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# Make the GET request to the Strava API
response = requests.get(athlete_url, headers=headers)

# Check if the request was successful and print the response
if response.status_code == 200:
    pprint(response.json())
else:
    pprint(f"Error: {response.status_code} - {response.text}")


{'badge_type_id': 0,
 'bio': None,
 'city': None,
 'country': None,
 'created_at': '2020-07-21T05:11:27Z',
 'firstname': 'William',
 'follower': None,
 'friend': None,
 'id': 64341718,
 'lastname': 'Catt',
 'premium': False,
 'profile': 'https://graph.facebook.com/2619742961675163/picture?height=256&width=256',
 'profile_medium': 'https://graph.facebook.com/2619742961675163/picture?height=256&width=256',
 'resource_state': 2,
 'sex': 'M',
 'state': None,
 'summit': False,
 'updated_at': '2023-09-25T00:56:44Z',
 'username': 'cattw',
 'weight': None}


Name Display, with profile picture and flag as a sort of header?

All other details seem irellevant, gender flag but don't display(?)
>

In [12]:


activites_url = "https://www.strava.com/api/v3/athlete/activities"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

params = {"per_page": 10, "page": 1}

activities_response = requests.get(activites_url, headers=headers, params=params)

if activities_response.status_code == 200:
    activities = activities_response.json()
    # Print the full JSON nicely
    import json
    print(json.dumps(activities, indent=2))
else:
    print(f"Error: {activities_response.status_code} - {activities_response.text}")


Error: 401 - {"message":"Authorization Error","errors":[{"resource":"Athlete","field":"access_token","code":"invalid"}]}
